## 실시간 엔드포인트를 DJL Serving을 이용해 호스팅
이번 노트북에서는 [Deep Java Library](https://djl.ai/) (DJL Serving)가 지원하는 대형 모델 추론 컨테이너(LMI)를 모델 서빙 솔루션으로 사용합니다. 
DJL Serving은 [transformers-neuronx](https://github.com/aws-neuron/transformers-neuronx) 라이브러리와 통합되어 있고, 모델 서버에 Neuron SDK가 미리 포함되어 있어 컴파일 작업을 사전 수행할 수 있습니다. 또한 뉴런 코어 간의 텐서 병렬 처리를 지원하고 있어 여러 뉴런 코어에 걸쳐 모델을 병렬로 로드할 수 있습니다. 이 노트북은 Amazon Elastic Compute Cloud(Amazon EC2) inf2.24xlarge 인스턴스에 Llama 3 모델을 배포합니다. 

In [ ]:
# 원하는 리전으로 변경합니다.
aws_region = "ap-northeast-1"

# 컨테이너 이미지 확인
image_uri = f"763104351884.dkr.ecr.{aws_region}.amazonaws.com/djl-inference:0.28.0-neuronx-sdk2.18.2"
image_uri

### Llama3 모델 다운로드
Llama3 모델을 사용하기 위한 첫 단계는 모델을 다운로드하는 것입니다. 이 과정은 Hugging Face Hub에서 제공하는 편리한 도구들을 활용하여 수행할 수 있습니다. 먼저, Hugging Face Hub에 로그인해야 합니다. 이는 huggingface_hub 라이브러리의 login() 함수를 통해 간단히 수행할 수 있습니다. 셸 실행 후 read 권한이 있는 token을 입력합니다. huggingface에서 token을 발급 받는 방법은 [여기](https://huggingface.co/docs/hub/security-tokens)를 참고하세요.

In [ ]:
# Hugging Face 로그인
from huggingface_hub import login, snapshot_download
login()

이 명령을 실행하면 로그인 창이 열리고 Hugging Face 계정으로 로그인할 수 있습니다. 로그인 과정은 모델에 접근하기 위한 인증을 제공합니다.
토큰 인증이 완료되면 이제 Llama3 모델을 다운로드할 차례입니다. 이를 위해 `snapshot_download()` 함수를 사용합니다.

In [ ]:
# 모델 다운로드
model_id = "meta-llama/Meta-Llama-3-8B"
local_model_path = "./models/llama3"
snapshot_download(repo_id=model_id, local_dir=local_model_path)

### 모델 서빙 아티팩트 준비하기
LMI 컨테이너는 Amazon S3 또는 Hugging Face Hub에 등록된 모델, 그리고 로컬 모델을 로드하도록 설정할 수 있습니다. 모델을 로드하고 호스팅하기 위해 serving.properties 파일에 필요한 파라미터들이 필요합니다. 구성 가능한 파라미터의 전체 목록은 [Transformers-NeuronX Engine in LMI 페이지의 serving.properties 항목](https://docs.djl.ai/docs/serving/serving/docs/lmi/user_guides/tnx_user_guide.html#servingproperties)을 참조하세요.

In [ ]:
# 모델 서버에 필요한 serving.properties 파일 생성 (로컬 경로 참조)

file_content = f"""engine=Python
option.entryPoint=djl_python.transformers_neuronx
option.model_id=/opt/ml/model/llama3
option.n_positions=8192
option.rolling_batch=vllm
option.neuron_optimize_level=1
option.max_rolling_batch_size=16
option.tensor_parallel_degree=8
option.dtype=fp16
option.enable_mixed_precision_accumulation=true
option.load_split_model=false
option.fuse_qkv=true
option.attention_layout=BSH
option.load_in_8bit=true
option.group_query_attention=replicated-heads
"""

with open("serving.properties","w") as f:
    f.write(file_content)

`serving.properties`를 models/llama3 디렉토리로 이동시킵니다.

In [ ]:
%%sh

# `serving.properties`를 models/llama3 디렉토리로 이동
mkdir -p logs
mkdir -p neuron-cache
cp serving.properties models/llama3/

다음으로, 앞서 정의한 모델 설정을 사용하여 Docker를 이용한 엔드포인트를 생성합니다. 모델 배포는 보통 4-5분 정도 소요되며, 이 과정에서 모델이 컴파일됩니다.

In [ ]:
!aws ecr get-login-password --region ap-northeast-1 | docker login --username AWS --password-stdin 763104351884.dkr.ecr.ap-northeast-1.amazonaws.com

In [ ]:
!docker run -it --rm --network=host \
  -v $(pwd)/models:/opt/ml/model/ \
  -v $(pwd)/logs:/opt/djl/logs \
  -v $(pwd)/neuron-cache:/var/tmp/neuron-compile-cache \
  -u djl \
  --device /dev/neuron0 \
  --device /dev/neuron1 \
  --device /dev/neuron2 \
  --device /dev/neuron3 \
  --device /dev/neuron4 \
  --device /dev/neuron5 \
  -e MODEL_LOADING_TIMEOUT=7200 \
  -e PREDICT_TIMEOUT=360 \
  {image_uri} serve

### 추론 테스트
Docker 엔드포인트가 생성된 후, curl 명령어를 사용하여 Docker 엔드포인트에 대해 실시간 예측을 수행할 수 있습니다.

* 추론 요청을 제출하고 응답을 받기 위해 curl 명령어를 사용합니다.
* 요청과 응답은 JSON 형식으로 이루어집니다.

별도의 터미널을 열고 모델 서버에 아래 명령어를 입력하고 추론 결과를 받아봅시다.

In [ ]:
curl -N -X POST "http://127.0.0.1:8080/predictions/llama3" \
     -H 'Content-Type: application/json' \
     -d '{
         "seq_length": 512,
         "inputs": "Welcome to Amazon Elastic Compute Cloud",
         "parameters": {
             "max_new_tokens": 32,
             "do_sample": "true"
         }
     }'